In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm
from datetime import datetime
import pytz
import json
import joblib 
import os
import optuna

/home/carolima/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v13.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v13.csv")



In [3]:
# If necessary change this part to test the model before the training process
df = challenge_set_updated.iloc[:,:]
# df = challenge_set_updated.sample(frac=0.001)

# Separating features and target variable
X = df.drop('tow', axis=1)
y = df['tow']



In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Handle categorical columns by Label Encoding
label_encoder = LabelEncoder()

# Specify the columns that are categorical
categorical_cols = ['adep', 'ades', 'aircraft_type', 'wtc', 'airline', 'offblock_season', 
                    'flight_duration_category', 'adep_region', 'ades_region', 'flight_direction', 
                    'Manufacturer', 'Model_FAA', 'Physical_Class_Engine', 'FAA_Weight']

# Convert these categorical columns into numerical form using Label Encoding
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Now, you can proceed with training LightGBM with this processed data


In [5]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import optuna

# Split the data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)


In [6]:

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor


# Define the objective function
def objective(trial):
    # Sample hyperparameters for LightGBM
    lgb_params = {
        'n_estimators': trial.suggest_int('lgb_n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('lgb_learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('lgb_max_depth', 3, 10),
        'num_leaves': trial.suggest_int('lgb_num_leaves', 31, 256),
    }

    # Sample hyperparameters for XGBoost
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 10),
        'subsample': trial.suggest_float('xgb_subsample', 0.5, 1.0),
    }

    # Sample hyperparameters for CatBoost
    cat_params = {
        'n_estimators': trial.suggest_int('cat_n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('cat_learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('cat_depth', 3, 10),
    }

    # Initialize the models with sampled hyperparameters
    lgb_model = LGBMRegressor(**lgb_params)
    xgb_model = XGBRegressor(**xgb_params, objective='reg:squarederror')
    cat_model = CatBoostRegressor(**cat_params, verbose=0)

    # Ensemble the models using VotingRegressor
    ensemble_model = VotingRegressor(
        estimators=[
            ('lgb', lgb_model),
            ('xgb', xgb_model),
            ('cat', cat_model)
        ]
    )

    # Split the dataset (use your own dataset instead of X, y)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the ensemble model
    ensemble_model.fit(X_train, y_train)

    # Make predictions
    y_pred = ensemble_model.predict(X_valid)

    # Calculate RMSE (you can use any other metric like R^2, MAE, etc.)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    
    return rmse

In [7]:
# Create an Optuna study object and optimize
study = optuna.create_study(direction='minimize')  # 'minimize' for minimizing RMSE
study.optimize(objective, n_trials=100)

# Print the best trial and its hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value (RMSE): {trial.value}")
print(f"  Params: {trial.params}")


[I 2024-10-07 21:48:58,110] A new study created in memory with name: no-name-5412f75b-10c3-49c0-be78-0d224ae59f32


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014689 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16312
[LightGBM] [Info] Number of data points in the train set: 295210, number of used features: 95
[LightGBM] [Info] Start training from score 79405.503419
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-07 21:49:50,953] Trial 0 finished with value: 3500.051924339308 and parameters: {'lgb_n_estimators': 289, 'lgb_learning_rate': 0.11043572369717504, 'lgb_max_depth': 3, 'lgb_num_leaves': 80, 'xgb_n_estimators': 229, 'xgb_learning_rate': 0.07628760668532356, 'xgb_max_depth': 6, 'xgb_subsample': 0.5016879814887788, 'cat_n_estimators': 98, 'cat_learning_rate': 0.09050552733027462, 'cat_depth': 5}. Best is trial 0 with value: 3500.051924339308.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016217 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16312
[LightGBM] [Info] Number of data points in the train set: 295210, number of used features: 95
[LightGBM] [Info] Start training from score 79405.503419
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-07 21:50:09,527] Trial 1 finished with value: 3453.5373526075923 and parameters: {'lgb_n_estimators': 67, 'lgb_learning_rate': 0.22903713600598577, 'lgb_max_depth': 4, 'lgb_num_leaves': 83, 'xgb_n_estimators': 295, 'xgb_learning_rate': 0.21730566945056626, 'xgb_max_depth': 3, 'xgb_subsample': 0.8756330731132735, 'cat_n_estimators': 88, 'cat_learning_rate': 0.16495092079676027, 'cat_depth': 6}. Best is trial 1 with value: 3453.5373526075923.


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015106 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16312
[LightGBM] [Info] Number of data points in the train set: 295210, number of used features: 95
[LightGBM] [Info] Start training from score 79405.503419
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-07 23:48:49,833] Trial 98 finished with value: 2605.5374856075737 and parameters: {'lgb_n_estimators': 135, 'lgb_learning_rate': 0.2989195156650173, 'lgb_max_depth': 9, 'lgb_num_leaves': 155, 'xgb_n_estimators': 221, 'xgb_learning_rate': 0.16189585898874137, 'xgb_max_depth': 9, 'xgb_subsample': 0.9301334580361513, 'cat_n_estimators': 207, 'cat_learning_rate': 0.2196446041828127, 'cat_depth': 10}. Best is trial 97 with value: 2595.6289821026758.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015873 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16312
[LightGBM] [Info] Number of data points in the train set: 295210, number of used features: 95
[LightGBM] [Info] Start training from score 79405.503419
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-07 23:50:35,517] Trial 99 finished with value: 2619.0124054930952 and parameters: {'lgb_n_estimators': 130, 'lgb_learning_rate': 0.2942159092898099, 'lgb_max_depth': 9, 'lgb_num_leaves': 152, 'xgb_n_estimators': 222, 'xgb_learning_rate': 0.16285539357880974, 'xgb_max_depth': 9, 'xgb_subsample': 0.8653852027681905, 'cat_n_estimators': 197, 'cat_learning_rate': 0.20990280246699694, 'cat_depth': 10}. Best is trial 97 with value: 2595.6289821026758.


Best trial:
  Value (RMSE): 2595.6289821026758
  Params: {'lgb_n_estimators': 128, 'lgb_learning_rate': 0.2981055698252218, 'lgb_max_depth': 9, 'lgb_num_leaves': 152, 'xgb_n_estimators': 248, 'xgb_learning_rate': 0.16074438652685877, 'xgb_max_depth': 9, 'xgb_subsample': 0.863380869809525, 'cat_n_estimators': 216, 'cat_learning_rate': 0.22268180851950062, 'cat_depth': 10}


In [8]:
# Use the best hyperparameters to create models
best_params = study.best_trial.params

# Initialize models with the best parameters
lgb_model = LGBMRegressor(
    n_estimators=best_params['lgb_n_estimators'],
    learning_rate=best_params['lgb_learning_rate'],
    max_depth=best_params['lgb_max_depth'],
    num_leaves=best_params['lgb_num_leaves']
)

xgb_model = XGBRegressor(
    n_estimators=best_params['xgb_n_estimators'],
    learning_rate=best_params['xgb_learning_rate'],
    max_depth=best_params['xgb_max_depth'],
    subsample=best_params['xgb_subsample'],
    objective='reg:squarederror'
)

cat_model = CatBoostRegressor(
    n_estimators=best_params['cat_n_estimators'],
    learning_rate=best_params['cat_learning_rate'],
    depth=best_params['cat_depth'],
    verbose=0
)

# Ensemble the models
ensemble_model = VotingRegressor(
    estimators=[
        ('lgb', lgb_model),
        ('xgb', xgb_model),
        ('cat', cat_model)
    ]
)

# Train the ensemble model on the entire dataset
ensemble_model.fit(X, y)

print('done')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025340 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16314
[LightGBM] [Info] Number of data points in the train set: 369013, number of used features: 95
[LightGBM] [Info] Start training from score 79482.257206
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
done


In [9]:

# Make predictions
y_pred = ensemble_model.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))


print(f"Best Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")


Best Model Performance - R^2 Score: 0.9987, RMSE: 1926.2284


In [17]:
import joblib

# Save the ensemble model to a file
model_filename = 'ensemble_model.pkl'
joblib.dump(ensemble_model, model_filename)

print(f"Model saved to {model_filename}")


Model saved to ensemble_model.pkl


In [18]:
from sklearn.preprocessing import LabelEncoder

submission_set_features = submission_set_updated.iloc[:,:-1]


In [19]:
submission_set_features = submission_set_features.dropna()


In [20]:
# Align submission_set with submission_set_features
submission_set = submission_set.loc[submission_set_features.index]


In [21]:
assert len(submission_set_features) == len(submission_set), "Mismatch in lengths"


In [22]:

# Identify columns with 'object' type that need to be converted
object_columns = submission_set_features.select_dtypes(include=['object']).columns

# Apply Label Encoding or One-Hot Encoding to these columns
# Option 1: Label Encoding
label_encoders = {}
for col in object_columns:
    le = LabelEncoder()
    submission_set_features[col] = le.fit_transform(submission_set_features[col])
    label_encoders[col] = le

# Alternatively, you could use One-Hot Encoding for better results
# submission_set_features = pd.get_dummies(submission_set_features, columns=object_columns)

# Ensure all columns are of numeric type
assert submission_set_features.dtypes.apply(lambda x: np.issubdtype(x, np.number)).all()

# Now you can use the model to make predictions
submission_set['tow'] = ensemble_model.predict(submission_set_features)
print(submission_set)


        flight_id        date                          callsign  adep  \
0       248753821  2022-01-01  3b3de0f3ad0ee192513995c02f7bf7cf  LTFJ   
1       248753822  2022-01-01  e06dd03d4a879ca37d9e18c1bd7cad16  EBBR   
2       248754498  2022-01-01  2d3b1c962c78c4ebeef11bcd51b9e94c  KMIA   
3       248757623  2022-01-01  81564432d3ee97c4bdf4cd8f006753dc  EGCN   
4       248763603  2022-01-01  84be079d7e660db105d91f600b4b3d59  EIDW   
...           ...         ...                               ...   ...   
105954  258066302  2022-12-31  2d3b4446c4d05a25196a9d52cab936fb  LTFJ   
105955  258068609  2022-12-31  253fd692ed441fac523081471c067772  LOWW   
105956  258068876  2022-12-31  c9fca302ca2e28acab0eb0bb1b46f11b  LTFM   
105957  258064675  2022-12-31  00f96ad0e382476649574ba044c764fc  EHAM   
105958  258058370  2022-12-31  5f0c222c7f7ceff3fbe75c854cce74c9  UBBB   

                     name_adep country_code_adep  ades          name_ades  \
0       Istanbul Sabiha Gokcen                

In [23]:
import os
from datetime import datetime

# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Define a timestamp for the file name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")

# Assuming submission_set is a DataFrame, save it to CSV
submission_set.to_csv(submission_file, index=False)

print(f"Submission saved to {submission_file}")


Submission saved to submissions/submission_20241008_104126.csv
